In [1]:
from transformers import T5ForConditionalGeneration as AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from peft import *
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup

cache_dir='/var/tmp/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
model_name = 'google-t5/t5-small'
#model_name = 'bigscience/bloomz-560m'

tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = AutoModel.from_pretrained(model_name,cache_dir=cache_dir)
print(model)
print(model.config)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [266]:
with torch.no_grad():
    #x = tokenizer("translate English to German: My name is Sarah and I live in London",return_tensors='pt')
    x = tokenizer("what is the label of this tweet? My name is Sarah and I live in London",return_tensors='pt')
    
    #x = tokenizer("Translate to English: Je t'aime. I",return_tensors='pt')
    
    #x['decoder_input_ids'] = torch.tensor([[tokenizer.pad_token_id]]) #x['input_ids']
    print(x)
    o = model.forward(**x ,decoder_input_ids= torch.tensor([[tokenizer.pad_token_id]]))
    print("forard: ",o.logits.shape)
    m = o.logits.argmax(-1)[0]
    print(tokenizer.decode(m))
    o = model.generate(**x)
    print("generate:" ,o.shape)
    print(tokenizer.decode(o[0]))
    
    

{'input_ids': tensor([[  125,    19,     8,  3783,    13,    48, 10657,    58,   499,   564,
            19,  8077,    11,    27,   619,    16,  1524,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
forard:  torch.Size([1, 1, 32128])
Label
generate: torch.Size([1, 6])
<pad> Label: no complaint</s>


In [4]:
# num_virtual_tokens important
peft_config = LoraConfig(task_type= TaskType.CAUSAL_LM )
#peft_config = PrefixTuningConfig(task_type="CAUSAL_LM", num_virtual_tokens=20)
#peft_config = PromptEncoderConfig(task_type="CAUSAL_LM", num_virtual_tokens=20, encoder_hidden_size=128)
"""peft_config = PromptTuningConfig(task_type= TaskType.CAUSAL_LM ,
                                         tokenizer_name_or_path=model_name,
                                         prompt_tuning_init=PromptTuningInit.TEXT,
                                         prompt_tuning_init_text='classify:',
                                         num_virtual_tokens=8))
                                    
"""
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


In [5]:
dataset_name = "twitter_complaints"
dataset = datasets.load_dataset("ought/raft", dataset_name,cache_dir=cache_dir)
print(dataset)
print(dataset["train"][0])
print(LABEL_TO_TEXT:=dataset["train"].features["Label"].names)


DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})
{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}
['Unlabeled', 'complaint', 'no complaint']


In [6]:
def pp(x):
    tokenized_x = tokenizer(f"what is the label of this tweet?\n{ x['Tweet text']}",add_special_tokens=True)#.to('cuda')
    tokenized_label = tokenizer(f" Label: {LABEL_TO_TEXT[x['Label']]}"  )
    #print(tokenized_x)

    model_input={}
    model_label={}
    model_input['input_ids'] = tokenized_x['input_ids'] #+ tokenized_label['input_ids']
    model_input['attention_mask'] = [1] * len(model_input['input_ids'] )
    #model_input['labels'] = [-100] * (len(tokenized_x['input_ids'])) + tokenized_label['input_ids']
    model_input['labels'] = tokenized_label['input_ids']
    
    
    return model_input

tokenized_train_dataset = dataset['train'].map(pp,remove_columns=dataset['train'].column_names)
print(tokenized_train_dataset)
tokenizer.decode( tokenized_train_dataset[0]['input_ids'])

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 50
})


'what is the label of this tweet? @HMRCcustomers No this is my first job</s>'

In [7]:
dcwp = DataCollatorWithPadding(tokenizer=tokenizer,padding=True,max_length=512)
#dcwp = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False,)

dcwp(tokenized_train_dataset[0:4])

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [8]:
import evaluate
import numpy as np
def compute_metric(output):
    inp , label = output
    #print(inp.shape,label.shape)
    #print(output)
    return {'acc':0}

batch_size = 1
trainer = Trainer(model,
        #data_collator=dcwp,
        train_dataset=tokenized_train_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metric,
        args=TrainingArguments(
                output_dir=output_dir,
                save_total_limit=1,
                num_train_epochs=50,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                
                
                
                
        ))

x = tokenized_train_dataset.select(range(1))
print(x[0])
print(len(x[0]['input_ids']))
print(len(x[0]['attention_mask']))
print(len(x[0]['labels']))

print(trainer.evaluate(x))
trainer.train()

{'input_ids': [125, 19, 8, 3783, 13, 48, 10657, 58, 3320, 566, 27396, 25697, 277, 465, 48, 19, 82, 166, 613, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [16229, 10, 150, 10394, 1]}
20
20
5
[2024-06-21 21:35:20,023] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


{'eval_loss': 8.42654800415039, 'eval_acc': 0, 'eval_runtime': 1.489, 'eval_samples_per_second': 0.672, 'eval_steps_per_second': 0.672}


Step,Training Loss
500,7.349900
1000,3.245100
1500,1.310900
2000,0.612300
2500,0.505500


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: F

TrainOutput(global_step=2500, training_loss=2.604756170654297, metrics={'train_runtime': 42.18, 'train_samples_per_second': 59.27, 'train_steps_per_second': 59.27, 'total_flos': 24708213964800.0, 'train_loss': 2.604756170654297, 'epoch': 50.0})

In [258]:
#model = model.to('cpu')
import random
#torch.manual_seed(7)
#random.seed(42)
#np.random.seed(42)
model.eval()
with torch.no_grad():
   print(dataset['train'][0])
   mi = tokenizer(f"what is the label of this tweet?\nI hate my job.",return_tensors='pt')['input_ids']
   print(tokenizer.decode(tokenized_train_dataset[0]['input_ids']))
   #mi = torch.tensor( tokenized_train_dataset[0:1]['input_ids'] )
   print(mi)
   #for i in range(10):
   o =  model.generate( input_ids=mi, max_new_tokens=10)#, attention_mask  = mi['attention_mask'])
   #print(o,o.shape)
   
   o = tokenizer.decode(o[0])
   print(o)

   

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}
what is the label of this tweet? @HMRCcustomers No this is my first job</s>
tensor([[  125,    19,     8,  3783,    13,    48, 10657,    58,    27,  5591,
            82,   613,     5,     1]])
<pad> Label: no complaint</s>
